In [10]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,load_model, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.noise import GaussianNoise
from keras.layers.normalization import BatchNormalization
import pandas as pd
import matplotlib.pyplot as plt
import time, pickle
from keras.utils import to_categorical
%matplotlib inline

In [12]:
num_classes = 10
class_name = {
    0: 'airplane',
    1: 'automobile',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck',
}
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
y_train = y_train.reshape(y_train.shape[0])
y_test = y_test.reshape(y_test.shape[0])

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'training samples')
print(x_test.shape[0], 'validation samples')

x_train shape: (50000, 32, 32, 3)
50000 training samples
10000 validation samples


In [3]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

In [4]:
y_train = to_categorical(y_train, nb_classes)
y_test = to_categorical(y_test, nb_classes)

In [14]:
# 定義輸入
input_shape = (32, 32, 3) # RGB影像224x224 (height, width, channel)

# 輸入層
img_input = Input(shape=input_shape, name='img_input')

# 第1個卷積區塊 (block1)
x = Conv2D(64, (3, 3), padding='same', activation='relu', name='block1_conv1',kernel_initializer='he_normal')(img_input)
x = Conv2D(64, (3, 3), padding='same', activation='relu', name='block1_conv2',kernel_initializer='he_normal')(x)
x = MaxPool2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# 第2個卷積區塊 (block2)
x = Conv2D(128, (3, 3), padding='same', activation='relu', name='block2_conv1',kernel_initializer='he_normal')(x)
x = Conv2D(128, (3, 3), padding='same', activation='relu', name='block2_conv2',kernel_initializer='he_normal')(x)
x = MaxPool2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# 第3個卷積區塊 (block3)
x = Conv2D(256, (3, 3), padding='same', activation='relu', name='block3_conv1',kernel_initializer='he_normal')(x)
x = Conv2D(256, (3, 3), padding='same', activation='relu', name='block3_conv2',kernel_initializer='he_normal')(x)
x = Conv2D(256, (3, 3), padding='same', activation='relu', name='block3_conv3',kernel_initializer='he_normal')(x)
x = MaxPool2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# 第4個卷積區塊 (block4)
x = Conv2D(512, (3, 3), padding='same', activation='relu', name='block4_conv1',kernel_initializer='he_normal')(x)
x = Conv2D(512, (3, 3), padding='same', activation='relu', name='block4_conv2',kernel_initializer='he_normal')(x)
x = Conv2D(512, (3, 3), padding='same', activation='relu', name='block4_conv3',kernel_initializer='he_normal')(x)
x = MaxPool2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# 第5個卷積區塊 (block5)
x = Conv2D(512, (3, 3), padding='same', activation='relu', name='block5_conv1',kernel_initializer='he_normal')(x)
x = Conv2D(512, (3, 3), padding='same', activation='relu', name='block5_conv2',kernel_initializer='he_normal')(x)
x = Conv2D(512, (3, 3), padding='same', activation='relu', name='block5_conv3',kernel_initializer='he_normal')(x)
x = MaxPool2D((2, 2), strides=(2, 2), name='block5_pool')(x)

# 前饋全連接區塊
x = Flatten(name='flatten')(x)
x = Dense(4096, activation='relu', name='fc1',kernel_initializer='he_normal')(x)
x = Dense(4096, activation='relu', name='fc2',kernel_initializer='he_normal')(x)
x = Dense(10, activation='softmax', name='predictions',kernel_initializer='he_normal')(x)

# 產生模型
model = Model(inputs=img_input, outputs=x, name='vgg16-funcapi')

# 打印網絡結構
model.summary()

Model: "vgg16-funcapi"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img_input (InputLayer)       [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)       

In [ ]:
nb_epoch = 100
batch_size = 256
start = time.time()
h = model1.fit(x_train, y_train, batch_size=batch_size, epochs=nb_epoch, validation_data=(x_test, y_test), shuffle=True)
model1.save('CIFAR10_model_no_data_augmentation.h5')
print('@ Total Time Spent: %.2f seconds' % (time.time() - start))